## Regressão Linear e Polinomial para Preços de Abóboras - Aula 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infográfico por Dasani Madipalli</figcaption>


#### Introdução

Até agora, exploraste o que é regressão com dados de exemplo recolhidos do conjunto de dados sobre preços de abóboras que utilizaremos ao longo desta aula. Também já os visualizaste usando `ggplot2`. 💪

Agora estás pronto para aprofundar o tema da regressão para ML. Nesta aula, vais aprender mais sobre dois tipos de regressão: *regressão linear básica* e *regressão polinomial*, juntamente com alguns dos conceitos matemáticos subjacentes a estas técnicas.

> Ao longo deste currículo, assumimos um conhecimento mínimo de matemática e procuramos torná-la acessível para estudantes de outras áreas. Por isso, presta atenção às notas, 🧮 destaques, diagramas e outras ferramentas de aprendizagem que ajudam na compreensão.

#### Preparação

Como lembrete, estás a carregar estes dados para fazer perguntas sobre eles.

-   Qual é a melhor altura para comprar abóboras?

-   Qual o preço que posso esperar por uma caixa de abóboras miniatura?

-   Devo comprá-las em cestos de meio alqueire ou em caixas de 1 1/9 alqueire? Vamos continuar a explorar estes dados.

Na aula anterior, criaste um `tibble` (uma reinterpretação moderna do data frame) e preencheste-o com parte do conjunto de dados original, padronizando os preços por alqueire. Ao fazer isso, no entanto, só conseguiste recolher cerca de 400 pontos de dados e apenas para os meses de outono. Talvez possamos obter um pouco mais de detalhe sobre a natureza dos dados ao limpá-los melhor? Vamos ver... 🕵️‍♀️

Para esta tarefa, vamos precisar dos seguintes pacotes:

-   `tidyverse`: O [tidyverse](https://www.tidyverse.org/) é uma [coleção de pacotes R](https://www.tidyverse.org/packages) projetada para tornar a ciência de dados mais rápida, fácil e divertida!

-   `tidymodels`: O [tidymodels](https://www.tidymodels.org/) é uma [estrutura de pacotes](https://www.tidymodels.org/packages/) para modelagem e aprendizagem automática.

-   `janitor`: O [pacote janitor](https://github.com/sfirke/janitor) oferece ferramentas simples para examinar e limpar dados desorganizados.

-   `corrplot`: O [pacote corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) fornece uma ferramenta visual exploratória para matrizes de correlação que suporta o reordenamento automático de variáveis para ajudar a detectar padrões ocultos entre variáveis.

Podes instalá-los da seguinte forma:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

O script abaixo verifica se tens os pacotes necessários para completar este módulo e instala-os para ti caso estejam em falta.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Vamos carregar estes pacotes incríveis e torná-los disponíveis na nossa sessão atual de R. (Isto é apenas para ilustração, `pacman::p_load()` já fez isso por você)

## 1. Uma linha de regressão linear

Como aprendeste na Lição 1, o objetivo de um exercício de regressão linear é conseguir traçar uma *linha* *de* *melhor ajuste* para:

-   **Mostrar relações entre variáveis**. Mostrar a relação entre as variáveis.

-   **Fazer previsões**. Fazer previsões precisas sobre onde um novo ponto de dados se situaria em relação a essa linha.

Para desenhar este tipo de linha, usamos uma técnica estatística chamada **Regressão dos Mínimos Quadrados**. O termo `mínimos quadrados` significa que todos os pontos de dados ao redor da linha de regressão são elevados ao quadrado e depois somados. Idealmente, essa soma final é o menor valor possível, porque queremos um número baixo de erros, ou `mínimos quadrados`. Assim, a linha de melhor ajuste é a linha que nos dá o menor valor para a soma dos erros ao quadrado - daí o nome *regressão dos mínimos quadrados*.

Fazemos isso porque queremos modelar uma linha que tenha a menor distância acumulada de todos os nossos pontos de dados. Também elevamos os termos ao quadrado antes de somá-los, pois estamos preocupados com a sua magnitude e não com a sua direção.

> **🧮 Mostra-me a matemática**
>
> Esta linha, chamada de *linha de melhor ajuste*, pode ser expressa por [uma equação](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` é a '`variável explicativa` ou `preditor`'. `Y` é a '`variável dependente` ou `resultado`'. A inclinação da linha é `b` e `a` é o ponto de interseção no eixo Y, que se refere ao valor de `Y` quando `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "slope = $y/x$")
    Infográfico por Jen Looper
>
> Primeiro, calcula a inclinação `b`.
>
> Em outras palavras, e referindo-nos à pergunta original sobre os dados das abóboras: "prever o preço de uma abóbora por alqueire ao longo dos meses", `X` referiria-se ao preço e `Y` ao mês de venda.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.pt.png)
    Infográfico por Jen Looper
> 
> Calcula o valor de Y. Se estás a pagar cerca de \$4, deve ser abril!
>
> A matemática que calcula a linha deve demonstrar a inclinação da linha, que também depende do ponto de interseção, ou onde `Y` está situado quando `X = 0`.
>
> Podes observar o método de cálculo destes valores no site [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Também visita [este calculador de mínimos quadrados](https://www.mathsisfun.com/data/least-squares-calculator.html) para ver como os valores dos números impactam a linha.

Não é tão assustador, certo? 🤓

#### Correlação

Outro termo importante para entender é o **Coeficiente de Correlação** entre as variáveis X e Y fornecidas. Usando um gráfico de dispersão, podes visualizar rapidamente este coeficiente. Um gráfico com pontos de dados alinhados de forma ordenada tem alta correlação, mas um gráfico com pontos de dados espalhados aleatoriamente entre X e Y tem baixa correlação.

Um bom modelo de regressão linear será aquele que tem um Coeficiente de Correlação alto (mais próximo de 1 do que de 0) usando o método de Regressão dos Mínimos Quadrados com uma linha de regressão.


## **2. Uma dança com dados: criar um data frame que será usado para modelação**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Ilustração por @allison_horst</figcaption>


<!--![Ilustração por \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.pt.jpg){width="700"}-->


Carregue as bibliotecas necessárias e o conjunto de dados. Converta os dados para um data frame contendo um subconjunto dos dados:

-   Obtenha apenas abóboras com preços por alqueire

-   Converta a data para um mês

-   Calcule o preço como uma média entre os preços altos e baixos

-   Converta o preço para refletir a quantidade por alqueire

> Abordámos estes passos na [lição anterior](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

No espírito de pura aventura, vamos explorar o [`pacote janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) que fornece funções simples para examinar e limpar dados desorganizados. Por exemplo, vamos dar uma olhada nos nomes das colunas dos nossos dados:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Podemos fazer melhor. Vamos tornar estes nomes de colunas `friendR` ao convertê-los para a convenção [snake_case](https://en.wikipedia.org/wiki/Snake_case) usando `janitor::clean_names`. Para saber mais sobre esta função: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Muito tidyR 🧹! Agora, uma dança com os dados usando `dplyr`, como na lição anterior! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Bom trabalho!👌 Agora tens um conjunto de dados limpo e organizado sobre o qual podes construir o teu novo modelo de regressão!

Que tal um gráfico de dispersão?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Um gráfico de dispersão lembra-nos que só temos dados mensais de agosto a dezembro. Provavelmente precisamos de mais dados para conseguir tirar conclusões de forma linear.

Vamos analisar novamente os nossos dados de modelação:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

E se quiséssemos prever o `price` de uma abóbora com base nas colunas `city` ou `package`, que são do tipo carácter? Ou, ainda mais simples, como poderíamos encontrar a correlação (que exige que ambos os seus inputs sejam numéricos) entre, por exemplo, `package` e `price`? 🤷🤷

Os modelos de aprendizagem automática funcionam melhor com características numéricas em vez de valores de texto, por isso, geralmente é necessário converter características categóricas em representações numéricas.

Isto significa que temos de encontrar uma forma de reformular os nossos preditores para os tornar mais fáceis de usar por um modelo de forma eficaz, um processo conhecido como `feature engineering`.


## 3. Pré-processamento de dados para modelação com recipes 👩‍🍳👨‍🍳

Atividades que reformulam os valores dos preditores para torná-los mais fáceis de usar de forma eficaz por um modelo são chamadas de `engenharia de características`.

Modelos diferentes têm requisitos de pré-processamento distintos. Por exemplo, mínimos quadrados requerem `codificação de variáveis categóricas`, como mês, variedade e city_name. Isso envolve simplesmente `traduzir` uma coluna com `valores categóricos` em uma ou mais `colunas numéricas` que substituem a original.

Por exemplo, suponha que os seus dados incluam a seguinte característica categórica:

|  cidade   |
|:---------:|
| Denver    |
| Nairobi   |
|  Tóquio   |

Pode aplicar *codificação ordinal* para substituir cada categoria por um valor inteiro único, assim:

| cidade |
|:------:|
|   0    |
|   1    |
|   2    |

E é isso que vamos fazer com os nossos dados!

Nesta secção, vamos explorar outro pacote incrível do Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - que foi criado para ajudar a pré-processar os seus dados **antes** de treinar o modelo. No seu núcleo, uma receita é um objeto que define quais passos devem ser aplicados a um conjunto de dados para prepará-lo para a modelação.

Agora, vamos criar uma receita que prepara os nossos dados para modelação, substituindo um número inteiro único por todas as observações nas colunas de preditores:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Incrível! 👏 Acabámos de criar a nossa primeira receita que especifica um resultado (preço) e os seus respetivos preditores, e que todas as colunas de preditores devem ser codificadas como um conjunto de números inteiros 🙌! Vamos analisar rapidamente:

-   A chamada para `recipe()` com uma fórmula indica à receita os *papéis* das variáveis, utilizando os dados de `new_pumpkins` como referência. Por exemplo, a coluna `price` foi atribuída ao papel de `outcome`, enquanto o resto das colunas foram atribuídas ao papel de `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` especifica que todos os preditores devem ser convertidos num conjunto de números inteiros, começando a numeração em 0.

Temos a certeza de que pode estar a pensar algo como: "Isto é tão fixe!! Mas e se eu precisar de confirmar que as receitas estão a fazer exatamente o que espero? 🤔"

Essa é uma ótima ideia! Veja, uma vez que a sua receita está definida, pode estimar os parâmetros necessários para realmente pré-processar os dados e, em seguida, extrair os dados processados. Normalmente, não precisa de fazer isto quando utiliza Tidymodels (vamos ver a convenção normal daqui a pouco-\> `workflows`), mas pode ser útil quando quiser fazer algum tipo de verificação para confirmar que as receitas estão a funcionar como esperado.

Para isso, vai precisar de mais dois verbos: `prep()` e `bake()`. E, como sempre, os nossos pequenos amigos do R criados por [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) ajudam a compreender isto melhor!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Arte por @allison_horst</figcaption>


<!--![Arte por \@allison_horst](../../../../../../translated_images/recipes.9ad10d8a4056bf89413fc33644924e0bd29d7c12fb2154e03a1ca3d2d6ea9323.pt.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): estima os parâmetros necessários a partir de um conjunto de treino que podem ser posteriormente aplicados a outros conjuntos de dados. Por exemplo, para uma determinada coluna preditora, qual observação será atribuída ao número inteiro 0, 1, 2, etc.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): utiliza uma receita preparada e aplica as operações a qualquer conjunto de dados.

Dito isto, vamos preparar e aplicar as nossas receitas para realmente confirmar que, nos bastidores, as colunas preditoras serão primeiro codificadas antes de ajustar um modelo.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Os dados processados `baked_pumpkins` têm todos os seus preditores codificados, confirmando que, de facto, os passos de pré-processamento definidos como a nossa receita funcionarão conforme esperado. Isto torna mais difícil para ti ler, mas muito mais inteligível para o Tidymodels! Dedica algum tempo a descobrir que observação foi mapeada para um número inteiro correspondente.

Também vale a pena mencionar que `baked_pumpkins` é um data frame no qual podemos realizar cálculos.

Por exemplo, vamos tentar encontrar uma boa correlação entre dois pontos dos teus dados para, potencialmente, construir um bom modelo preditivo. Vamos usar a função `cor()` para fazer isto. Escreve `?cor()` para saberes mais sobre a função.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Afinal, há apenas uma correlação fraca entre a Cidade e o Preço. No entanto, há uma correlação um pouco melhor entre o Pacote e o seu Preço. Faz sentido, certo? Normalmente, quanto maior a caixa de produtos, maior o preço.

Já que estamos nisso, vamos também tentar visualizar uma matriz de correlação de todas as colunas usando o pacote `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Muito melhor.

Uma boa pergunta a fazer agora com estes dados seria: '`Que preço posso esperar de um determinado pacote de abóboras?`' Vamos começar!

> Nota: Quando **`bake()`** a receita preparada **`pumpkins_prep`** com **`new_data = NULL`**, extrai os dados de treino processados (ou seja, codificados). Se tiver outro conjunto de dados, por exemplo, um conjunto de teste, e quiser ver como uma receita o pré-processaria, basta fazer o bake de **`pumpkins_prep`** com **`new_data = test_set`**.

## 4. Construir um modelo de regressão linear

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infográfico por Dasani Madipalli</figcaption>


<!--![Infográfico por Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.pt.png){width="800"}-->


Agora que criámos uma receita e confirmámos que os dados serão pré-processados de forma adequada, vamos agora construir um modelo de regressão para responder à pergunta: `Qual o preço que posso esperar de um determinado pacote de abóbora?`

#### Treinar um modelo de regressão linear utilizando o conjunto de treino

Como já deve ter percebido, a coluna *price* é a variável `resultado`, enquanto a coluna *package* é a variável `preditor`.

Para isso, primeiro vamos dividir os dados de forma que 80% sejam usados para treino e 20% para teste, depois definiremos uma receita que codificará a coluna preditora num conjunto de inteiros e, em seguida, construiremos uma especificação de modelo. Não vamos preparar e aplicar a nossa receita, pois já sabemos que ela irá pré-processar os dados conforme esperado.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Bom trabalho! Agora que temos uma receita e uma especificação de modelo, precisamos encontrar uma forma de agrupá-los num objeto que primeiro pré-processa os dados (prep+bake nos bastidores), ajusta o modelo nos dados pré-processados e também permite possíveis atividades de pós-processamento. Que tal para a tua tranquilidade!🤩

No Tidymodels, este objeto prático chama-se [`workflow`](https://workflows.tidymodels.org/) e guarda convenientemente os teus componentes de modelação! Isto é o que chamaríamos de *pipelines* em *Python*.

Então, vamos agrupar tudo num workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Além disso, um fluxo de trabalho pode ser ajustado/treinado de forma muito semelhante a um modelo.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

A partir dos resultados do modelo, podemos observar os coeficientes aprendidos durante o treino. Eles representam os coeficientes da linha de melhor ajuste que nos proporciona o menor erro geral entre a variável real e a prevista.

#### Avaliar o desempenho do modelo utilizando o conjunto de teste

É hora de ver como o modelo se saiu 📏! Como fazemos isso?

Agora que treinámos o modelo, podemos utilizá-lo para fazer previsões no test_set usando `parsnip::predict()`. Depois, podemos comparar estas previsões com os valores reais das etiquetas para avaliar quão bem (ou não!) o modelo está a funcionar.

Vamos começar por fazer previsões para o conjunto de teste e, em seguida, ligar as colunas ao conjunto de teste.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Sim, acabou de treinar um modelo e utilizou-o para fazer previsões! 🔮 Será que é bom? Vamos avaliar o desempenho do modelo!

No Tidymodels, fazemos isso utilizando `yardstick::metrics()`! Para regressão linear, vamos focar nas seguintes métricas:

-   `Root Mean Square Error (RMSE)`: A raiz quadrada do [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Esta métrica fornece um valor absoluto na mesma unidade que o rótulo (neste caso, o preço de uma abóbora). Quanto menor o valor, melhor o modelo (de forma simplista, representa o preço médio pelo qual as previsões estão erradas!).

-   `Coefficient of Determination (geralmente conhecido como R-quadrado ou R2)`: Uma métrica relativa em que, quanto maior o valor, melhor o ajuste do modelo. Essencialmente, esta métrica representa o quanto da variância entre os valores previstos e os valores reais o modelo consegue explicar.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Lá se vai o desempenho do modelo. Vamos ver se conseguimos obter uma indicação melhor ao visualizar um gráfico de dispersão do pacote e do preço, e depois usar as previsões feitas para sobrepor uma linha de melhor ajuste.

Isto significa que teremos de preparar e processar o conjunto de teste para codificar a coluna do pacote e, em seguida, associar isso às previsões feitas pelo nosso modelo.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Ótimo! Como pode ver, o modelo de regressão linear não generaliza muito bem a relação entre um pacote e o seu preço correspondente.

🎃 Parabéns, acabou de criar um modelo que pode ajudar a prever o preço de algumas variedades de abóboras. O seu campo de abóboras para o feriado será lindo. Mas provavelmente pode criar um modelo ainda melhor!

## 5. Construir um modelo de regressão polinomial

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infográfico por Dasani Madipalli</figcaption>


<!--![Infográfico por Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.pt.png){width="800"}-->


Às vezes, os nossos dados podem não ter uma relação linear, mas ainda assim queremos prever um resultado. A regressão polinomial pode ajudar-nos a fazer previsões para relações não lineares mais complexas.

Por exemplo, considere a relação entre o tamanho e o preço no nosso conjunto de dados de abóboras. Embora, por vezes, exista uma relação linear entre as variáveis - quanto maior o volume da abóbora, maior o preço - outras vezes estas relações não podem ser representadas como um plano ou uma linha reta.

> ✅ Aqui estão [alguns exemplos adicionais](https://online.stat.psu.edu/stat501/lesson/9/9.8) de dados que poderiam usar regressão polinomial
>
> Observe novamente a relação entre Variedade e Preço no gráfico anterior. Este diagrama de dispersão parece que deveria ser necessariamente analisado por uma linha reta? Talvez não. Neste caso, pode experimentar a regressão polinomial.
>
> ✅ Polinómios são expressões matemáticas que podem consistir em uma ou mais variáveis e coeficientes

#### Treinar um modelo de regressão polinomial usando o conjunto de treino

A regressão polinomial cria uma *linha curva* para ajustar melhor os dados não lineares.

Vamos ver se um modelo polinomial terá um desempenho melhor ao fazer previsões. Seguiremos um procedimento algo semelhante ao que fizemos anteriormente:

-   Criar uma receita que especifique os passos de pré-processamento que devem ser realizados nos nossos dados para os preparar para modelação, ou seja: codificar preditores e calcular polinómios de grau *n*

-   Construir uma especificação de modelo

-   Agrupar a receita e a especificação de modelo num fluxo de trabalho

-   Criar um modelo ajustando o fluxo de trabalho

-   Avaliar o desempenho do modelo nos dados de teste

Vamos começar!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Avaliar o desempenho do modelo

👏👏Construíste um modelo polinomial, vamos fazer previsões no conjunto de teste!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, vamos avaliar como o modelo se saiu no test_set usando `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Desempenho muito melhor.

O `rmse` diminuiu de cerca de 7 para cerca de 3, uma indicação de um erro reduzido entre o preço real e o preço previsto. Pode-se *interpretar de forma aproximada* que, em média, as previsões incorretas erram por cerca de 3€. O `rsq` aumentou de cerca de 0.4 para 0.8.

Todos estes indicadores mostram que o modelo polinomial tem um desempenho muito superior ao modelo linear. Bom trabalho!

Vamos ver se conseguimos visualizar isto!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Pode ver uma linha curva que se ajusta melhor aos seus dados! 🤩

Pode torná-la mais suave ao passar uma fórmula polinomial para `geom_smooth` assim:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Muito parecido com uma curva suave!🤩

Aqui está como fazer uma nova previsão:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


A previsão do `polynomial model` faz sentido, considerando os gráficos de dispersão de `price` e `package`! E, se este modelo for melhor do que o anterior, olhando para os mesmos dados, será necessário prever um orçamento para estas abóboras mais caras!

🏆 Muito bem! Você criou dois modelos de regressão em uma única lição. Na seção final sobre regressão, aprenderá sobre regressão logística para determinar categorias.

## **🚀Desafio**

Teste várias variáveis diferentes neste notebook para ver como a correlação corresponde à precisão do modelo.

## [**Questionário pós-aula**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Revisão & Estudo Autônomo**

Nesta lição aprendemos sobre Regressão Linear. Existem outros tipos importantes de Regressão. Leia sobre as técnicas Stepwise, Ridge, Lasso e Elasticnet. Um bom curso para aprender mais é o [curso de Aprendizagem Estatística de Stanford](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Se quiser aprender mais sobre como usar o incrível framework Tidymodels, consulte os seguintes recursos:

-   Website do Tidymodels: [Introdução ao Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn e Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **AGRADECIMENTOS A:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) por criar as ilustrações incríveis que tornam o R mais acolhedor e envolvente. Encontre mais ilustrações na sua [galeria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original no seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se uma tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes do uso desta tradução.
